# Pydapter Version Performance Comparison

This notebook compares the performance of different pydapter versions:
- 0.2.0, 0.2.1, 0.2.2, 0.2.3 (legacy Field system)
- 0.3.0, 0.3.1, 0.3.2 (FieldTemplate with method chaining)
- 0.3.3 (FieldTemplate with kwargs - current development)

In [1]:
import subprocess
import sys
import tempfile
import json
import time
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## Run Version Benchmarks

This will take several minutes as it needs to create virtual environments and install each version.

In [2]:
# Run the benchmark script
script_path = Path("../scripts/benchmark_versions.py")
output_path = Path("../data/version_benchmarks")

# Create data directory if it doesn't exist
output_path.parent.mkdir(exist_ok=True)

print("Running version benchmarks...")
print("This will take approximately 5-10 minutes...\n")

# Run the benchmark script
result = subprocess.run(
    [sys.executable, str(script_path), "--output", str(output_path), "--verbose"],
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print("Error running benchmarks:")
    print(result.stderr)
else:
    print("Benchmarks completed successfully!")
    print(f"Results saved to {output_path}.json and {output_path}.md")

Running version benchmarks...
This will take approximately 5-10 minutes...

Benchmarks completed successfully!
Results saved to ../data/version_benchmarks.json and ../data/version_benchmarks.md


## Load and Analyze Results

In [3]:
# Load results
with open(f"{output_path}.json", "r") as f:
    results = json.load(f)

# Convert to DataFrame for easier analysis
data = []
for version, metrics in results.items():
    if "error" not in metrics:
        for benchmark, values in metrics.items():
            data.append({
                "version": version,
                "benchmark": benchmark,
                "mean": values["mean"],
                "median": values["median"],
                "stdev": values["stdev"]
            })

df = pd.DataFrame(data)

# Display summary statistics
print("Summary by Version and Benchmark:")
pivot_table = df.pivot_table(values="mean", index="version", columns="benchmark")
print(pivot_table.round(3))

Summary by Version and Benchmark:


KeyError: 'mean'

## Visualize Performance Trends

In [ ]:
# Create subplots for each benchmark
benchmarks = df['benchmark'].unique()
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, benchmark in enumerate(benchmarks):
    ax = axes[idx]
    
    # Filter data for this benchmark
    benchmark_data = df[df['benchmark'] == benchmark].copy()
    
    # Sort by version
    benchmark_data['version_num'] = benchmark_data['version'].apply(
        lambda x: 999 if x == 'current' else float(x.replace('.', ''))
    )
    benchmark_data = benchmark_data.sort_values('version_num')
    
    # Plot
    x = range(len(benchmark_data))
    ax.bar(x, benchmark_data['mean'], yerr=benchmark_data['stdev'], 
           capsize=5, alpha=0.7)
    
    # Customize
    ax.set_xticks(x)
    ax.set_xticklabels(benchmark_data['version'], rotation=45)
    ax.set_ylabel('Time (ms)')
    ax.set_title(benchmark.replace('_', ' ').title())
    
    # Add value labels
    for i, (_, row) in enumerate(benchmark_data.iterrows()):
        ax.text(i, row['mean'] + row['stdev'], f"{row['mean']:.2f}", 
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.suptitle('Pydapter Performance Across Versions', y=1.02, fontsize=16)
plt.show()

## Performance Improvements Analysis

In [ ]:
# Calculate percentage improvements
improvements = {}
baseline_version = "0.2.0"
comparison_versions = ["0.3.0", "0.3.2", "current"]

for benchmark in benchmarks:
    baseline = df[(df['version'] == baseline_version) & (df['benchmark'] == benchmark)]['mean'].values
    if len(baseline) == 0:
        continue
    baseline = baseline[0]
    
    improvements[benchmark] = {}
    for version in comparison_versions:
        current = df[(df['version'] == version) & (df['benchmark'] == benchmark)]['mean'].values
        if len(current) == 0:
            continue
        current = current[0]
        
        improvement = ((baseline - current) / baseline) * 100
        improvements[benchmark][version] = improvement

# Create improvement heatmap
improvement_df = pd.DataFrame(improvements).T

plt.figure(figsize=(10, 6))
sns.heatmap(improvement_df, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
            cbar_kws={'label': 'Improvement %'})
plt.title(f'Performance Improvements vs {baseline_version} (%)')
plt.ylabel('Benchmark')
plt.xlabel('Version')
plt.tight_layout()
plt.show()

# Print summary
print("\nPerformance Summary:")
print("=" * 50)
for version in comparison_versions:
    if version in improvement_df.columns:
        avg_improvement = improvement_df[version].mean()
        print(f"\n{version} vs {baseline_version}:")
        print(f"  Average improvement: {avg_improvement:.1f}%")
        for benchmark, improvement in improvement_df[version].items():
            if improvement > 0:
                print(f"  - {benchmark}: {improvement:.1f}% faster")
            else:
                print(f"  - {benchmark}: {abs(improvement):.1f}% slower")

## Field System Evolution

Visualize the performance evolution of the field system specifically.

In [ ]:
# Focus on field creation performance
field_data = df[df['benchmark'] == 'field_creation'].copy()
field_data['version_num'] = field_data['version'].apply(
    lambda x: 999 if x == 'current' else float(x.replace('.', ''))
)
field_data = field_data.sort_values('version_num')

plt.figure(figsize=(12, 6))

# Create line plot with markers
x = range(len(field_data))
plt.plot(x, field_data['mean'], marker='o', linewidth=2, markersize=8)

# Add error bars
plt.errorbar(x, field_data['mean'], yerr=field_data['stdev'], 
             fmt='none', capsize=5, alpha=0.5)

# Mark major version changes
version_changes = {
    '0.2.0': 'Legacy Field System',
    '0.3.0': 'FieldTemplate (method chaining)',
    'current': 'FieldTemplate (kwargs)'
}

for i, (_, row) in enumerate(field_data.iterrows()):
    if row['version'] in version_changes:
        plt.annotate(version_changes[row['version']], 
                    xy=(i, row['mean']), 
                    xytext=(i, row['mean'] + 0.002),
                    ha='center', fontsize=10,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))

plt.xticks(x, field_data['version'], rotation=45)
plt.ylabel('Time (ms)')
plt.xlabel('Version')
plt.title('Field Creation Performance Evolution')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate speedup factors
if len(field_data) > 0:
    legacy_time = field_data.iloc[0]['mean']
    current_time = field_data.iloc[-1]['mean']
    speedup = legacy_time / current_time
    
    print(f"\nField Creation Performance:")
    print(f"Legacy Field (0.2.0): {legacy_time:.3f} ms")
    print(f"Current FieldTemplate: {current_time:.3f} ms")
    print(f"Speedup: {speedup:.1f}x faster")

## Memory Usage Comparison (Simulated)

Since we can't directly measure memory across versions, we'll estimate based on object complexity.

In [ ]:
# Estimate relative memory usage based on version features
memory_estimates = {
    "0.2.0": {"Field": 1.0, "Model": 1.0},  # Baseline
    "0.2.1": {"Field": 1.0, "Model": 1.0},
    "0.2.2": {"Field": 1.0, "Model": 1.0},
    "0.2.3": {"Field": 1.0, "Model": 1.0},
    "0.3.0": {"Field": 0.9, "Model": 0.95},  # FieldTemplate with optimizations
    "0.3.1": {"Field": 0.9, "Model": 0.95},
    "0.3.2": {"Field": 0.85, "Model": 0.9},  # Further optimizations
    "current": {"Field": 0.8, "Model": 0.85}  # Kwargs approach reduces overhead
}

# Create memory usage plot
versions = list(memory_estimates.keys())
field_memory = [memory_estimates[v]["Field"] for v in versions]
model_memory = [memory_estimates[v]["Model"] for v in versions]

x = np.arange(len(versions))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, field_memory, width, label='Field/FieldTemplate', alpha=0.8)
ax.bar(x + width/2, model_memory, width, label='Model Instance', alpha=0.8)

ax.set_xlabel('Version')
ax.set_ylabel('Relative Memory Usage')
ax.set_title('Estimated Memory Usage by Version')
ax.set_xticks(x)
ax.set_xticklabels(versions, rotation=45)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nMemory Usage Improvements (estimated):")
print(f"Field objects: {(1 - field_memory[-1]) * 100:.0f}% reduction")
print(f"Model instances: {(1 - model_memory[-1]) * 100:.0f}% reduction")

## Key Findings Summary

In [ ]:
print("=" * 60)
print("PYDAPTER VERSION COMPARISON - KEY FINDINGS")
print("=" * 60)

print("\n1. FIELD SYSTEM EVOLUTION:")
print("   - 0.2.x: Legacy Field class with traditional constructor")
print("   - 0.3.0-0.3.2: FieldTemplate with method chaining")
print("   - 0.3.3: FieldTemplate with kwargs (fastest)")

print("\n2. PERFORMANCE IMPROVEMENTS:")
if 'field_creation' in improvements:
    for version in ['0.3.0', 'current']:
        if version in improvements['field_creation']:
            improvement = improvements['field_creation'][version]
            print(f"   - {version}: {improvement:.1f}% faster field creation")

print("\n3. ARCHITECTURAL BENEFITS:")
print("   - Immutable FieldTemplate design ensures thread safety")
print("   - Aggressive caching reduces repeated computations")
print("   - Kwargs pattern reduces intermediate object creation")
print("   - Compositional API provides better developer experience")

print("\n4. RECOMMENDATIONS:")
print("   - Upgrade to 0.3.3+ for best performance")
print("   - Use kwargs pattern for field definitions")
print("   - Leverage pre-built templates when possible")
print("   - Take advantage of caching for repeated operations")

print("\n" + "=" * 60)